In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from datetime import date, timedelta

In [2]:
def convert_time(list_in):
    out = []
    for item in list_in:
        item = str(item)
        year = int(item[:4])
        mon = int(item[4:6])
        if len(item) >6:
            day = int(item[6:])
            new = date(year, mon, day)
        else:
            year += mon//12
            mon = mon%12 + 1
            new = date(year, mon, 1) - timedelta(1)
            while new.weekday()>=5:
                new = new - timedelta(1)
        out.append(new)
    return out

def Reconcile(list_in, Ref_list):
    list_out = deepcopy(list_in)
    Ref_List.sort()
    for i, days in enumerate(list_in):
        if days < Ref_list[0] or days > Ref_list[-1]:
            continue
        while days not in Ref_list:
            days = days - timedelta(1)
        list_out[i] = days
    return list_out

In [3]:
Daily = pd.read_csv('PredictorDataDaily.csv', index_col = 0)
Month_v0 = pd.read_csv('PredictorDataMonthly.csv', index_col = 0)
Month_v1 = pd.read_csv('Monthly_Data.csv', index_col = 0)

In [4]:
Unique = Daily[['Mkt-RF','SMB','HML']]
Unique.index = pd.to_datetime(convert_time(Unique.index))
Month_v0.index = pd.to_datetime(convert_time(Month_v0.index))
Month_v1.index = pd.to_datetime(Month_v1.index)

In [5]:
Ref_List = list(Unique.index)
Month_v1.index = pd.to_datetime(Reconcile(list(Month_v1.index), Ref_List))
Common = Unique.index.intersection(Month_v1.index)
Month_v1_subset = Month_v1.loc[Common]

In [6]:
Month_start = Unique.index[0]
Result = np.zeros([len(Common), 90]) ##Create a more than big enough numpy array to contain results
for i, checkpoint in enumerate(Common):
    Month_End = checkpoint
    Daily_Uniques = Unique.loc[Month_start:Month_End].unstack().values
    Result[i, :len(Daily_Uniques)] = Daily_Uniques
    Month_start = Month_End + timedelta(1)
All_Zero_idx = np.argwhere(np.all(Result[..., :] == 0, axis=0)) #Then delete the all-zero columns 
Result = np.delete(Result, All_Zero_idx, axis=1)

In [7]:
N_day = int(Result.shape[1]/3)
Name_list = []
for i in range(3):
    factor = Unique.columns[i]
    for j in range(N_day):
        Name = factor + ' day ' + str(j)
        Name_list.append(Name)

In [8]:
Month_v1_subset = pd.concat([Month_v1_subset, pd.DataFrame(Result, index = Common, columns = Name_list)], axis= 1)
Month_v1_subset.to_csv('Monthly_Data_Added_Daily.csv')

In [9]:
Ref_List = list(Unique.index)
Month_v0.index = pd.to_datetime(Reconcile(list(Month_v0.index), Ref_List))
Common = Unique.index.intersection(Month_v0.index)
Month_v0_subset = Month_v0.loc[Common]
Month_start = Unique.index[0]
Result = np.zeros([len(Common), 90]) ##Create a more than big enough numpy array to contain results
for i, checkpoint in enumerate(Common):
    Month_End = checkpoint
    Daily_Uniques = Unique.loc[Month_start:Month_End].unstack().values
    Result[i, :len(Daily_Uniques)] = Daily_Uniques
    Month_start = Month_End + timedelta(1)
All_Zero_idx = np.argwhere(np.all(Result[..., :] == 0, axis=0)) #Then delete the all-zero columns 
Result = np.delete(Result, All_Zero_idx, axis=1)
N_day = int(Result.shape[1]/3)
Name_list = []
for i in range(3):
    factor = Unique.columns[i]
    for j in range(N_day):
        Name = factor + ' day ' + str(j)
        Name_list.append(Name)
Month_v0_subset = pd.concat([Month_v0_subset, pd.DataFrame(Result, index = Common, columns = Name_list)], axis= 1)
Month_v0_subset.to_csv('PredictorDataMonthly_Added_Daily.csv')